# Part2

In [2]:
import json
from urllib.request import urlopen
import requests,json,os,sys,time,re
import pandas as pd
import numpy as np

In [13]:
def show_work_status(singleCount, totalCount, currentCount):
    currentCount+=singleCount
    percentage=1.*currentCount/totalCount*100
    status='>'*int(percentage/2)+'-'*(50-int(percentage/2))
    if percentage % 3:
        sys.stdout.write('\rStatus:[{0}]{1:.2f}%'.format(status,percentage))
        sys.stdout.flush()
    if percentage>=100: print('\n')

In [3]:
r_part2=requests.get('http://steamspy.com/api.php?request=top100forever')
r_part2.content

b'{"730":{"appid":730,"name":"Counter-Strike: Global Offensive","developer":"Valve, Hidden Path Entertainment","publisher":"Valve","score_rank":"","positive":3967641,"negative":548768,"userscore":0,"owners":"100,000,000 .. 200,000,000","average_forever":27122,"average_2weeks":1039,"median_forever":7741,"median_2weeks":398,"price":"0","initialprice":"0","discount":"0"},\n"570":{"appid":570,"name":"Dota 2","developer":"Valve","publisher":"Valve","score_rank":"","positive":1122167,"negative":201487,"userscore":0,"owners":"100,000,000 .. 200,000,000","average_forever":32998,"average_2weeks":1692,"median_forever":893,"median_2weeks":902,"price":"0","initialprice":"0","discount":"0"},\n"440":{"appid":440,"name":"Team Fortress 2","developer":"Valve","publisher":"Valve","score_rank":"","positive":638488,"negative":40287,"userscore":0,"owners":"50,000,000 .. 100,000,000","average_forever":9119,"average_2weeks":1163,"median_forever":395,"median_2weeks":181,"price":"0","initialprice":"0","discoun

In [4]:
dic_part2 = r_part2.json()   ##############  To make requests.models.Response become dict   ###############
dic_part2

{'730': {'appid': 730,
  'name': 'Counter-Strike: Global Offensive',
  'developer': 'Valve, Hidden Path Entertainment',
  'publisher': 'Valve',
  'score_rank': '',
  'positive': 3967641,
  'negative': 548768,
  'userscore': 0,
  'owners': '100,000,000 .. 200,000,000',
  'average_forever': 27122,
  'average_2weeks': 1039,
  'median_forever': 7741,
  'median_2weeks': 398,
  'price': '0',
  'initialprice': '0',
  'discount': '0'},
 '570': {'appid': 570,
  'name': 'Dota 2',
  'developer': 'Valve',
  'publisher': 'Valve',
  'score_rank': '',
  'positive': 1122167,
  'negative': 201487,
  'userscore': 0,
  'owners': '100,000,000 .. 200,000,000',
  'average_forever': 32998,
  'average_2weeks': 1692,
  'median_forever': 893,
  'median_2weeks': 902,
  'price': '0',
  'initialprice': '0',
  'discount': '0'},
 '440': {'appid': 440,
  'name': 'Team Fortress 2',
  'developer': 'Valve',
  'publisher': 'Valve',
  'score_rank': '',
  'positive': 638488,
  'negative': 40287,
  'userscore': 0,
  'owners

* Store the data we got as csv file

In [5]:
total_count= 0
with open('app_part2.csv','w') as part_two:
    for game_id,game_details in dic_part2.items():
        total_count =total_count+1
        part_two.write(json.dumps({game_id:game_details}))
        part_two.write('\n')

* Build a dict to store the data we got from http requests

In [14]:
dic_game_genre = {'appid':{},'name':{},'positive':{},'negative':{}}
current_count = 0
for game_id,game_details in dic_part2.items():
    if game_details!={} and game_details is not None:
        dic_game_genre['appid'].update({game_id:game_details.get("appid",{})})
        dic_game_genre['name'].update({game_id:game_details.get('name',{})})
        dic_game_genre['positive'].update({game_id:game_details.get('positive',{})})
        dic_game_genre['negative'].update({game_id:game_details.get('negative',{})})
    show_work_status(1,total_count,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [15]:
pos = list(dic_game_genre.get('positive').values())
neg = list(dic_game_genre.get('negative').values())
appid = list(dic_game_genre.get('appid').values())
name = list(dic_game_genre.get('name').values())

* Let's build a dataframe with Game_name, Game_id, number of positive reviews, number of negative reviews and each game's positive_rate

In [21]:
percentile_list = pd.DataFrame(
    {'Positive': pos,
     'Negative': neg,
     'Game_id': appid,
     'Game_name': name,
     #'Positive_Rate': percentile_list["Positive"]/(percentile_list["Positive"]+percentile_list["Negative"])
    })

percentile_list['Positive_Rate'] =  percentile_list["Positive"]/(percentile_list["Positive"]+percentile_list["Negative"])
percentile_list.sort_values("Positive_Rate", axis = 0, ascending = False, 
                 inplace = True, na_position ='first') 
percentile_list = percentile_list.reset_index(drop=True)
percentile_list

,Positive,Negative,appid,Game_name,Positive_Rate
0,195997,2472,620,Portal 2,0.987545
1,188807,3341,431960,Wallpaper Engine,0.982612
2,69635,1270,400,Portal,0.982089
3,380163,7393,292030,The Witcher 3: Wild Hunt,0.980924
4,537066,11971,105600,Terraria,0.978196
5,243832,5907,413150,Stardew Valley,0.976347
6,156776,4082,10,Counter-Strike,0.974624
7,308475,8366,227300,Euro Truck Simulator 2,0.973596
8,99773,2938,220,Half-Life 2,0.971395
9,352879,10596,550,Left 4 Dead 2,0.970848


* We already have games with highest positive review rate, let's see games which have most positive review rate although they may have many negative reviews.

In [19]:
print('Positive Rate Based Model')
df_genre_based_results = pd.Series(list(dic_game_genre.get('positive').values()),list(dic_game_genre.get('name').values())).sort_values(ascending = False)
                                       
df_genre_based_results.head(10)

Positive Rate Based Model


Counter-Strike: Global Offensive    3967641
Dota 2                              1122167
PLAYERUNKNOWN'S BATTLEGROUNDS        803481
Grand Theft Auto V                   715435
Team Fortress 2                      638488
Tom Clancy's Rainbow Six Siege       617489
Garry's Mod                          551242
Terraria                             537066
Rocket League                        391119
PAYDAY 2                             380915
dtype: int64